In [2]:
import json
import re
import base64
import numpy as np
import pandas as pd
from sklearn.decomposition import PCA
import plotly.express as px

import dotenv
dotenv.load_dotenv()

import openai
client = openai.OpenAI()

import tiktoken
enc = tiktoken.get_encoding("cl100k_base")

from MailInterface import getMessages

In [3]:
def filter_by_whitelist(text):
    ...

In [60]:
def extract_urls_with_context(text, N=20, W=200):
    """
    Extracts all URLs from the given text along with N tokens before and after each URL.

    :param text: The text from which to extract URLs.
    :param N: The number of tokens to include before and after each URL.
    :return: A list of tuples, each containing (URL, N tokens before, N tokens after).
    """
    whitelist = [
        'unsubscribe',
        'manage',
        'subscription'
    ]
    def getWeights(toks, direction, whitelist=whitelist):
        binary = []
        for t in toks:
            added = False
            for word in whitelist:
                # if t.lower() in word:
                # if t.lower().strip() == word.strip():
                if word in t.lower().strip():
                    binary.append(True)
                    added=True
                    break
            if not added:
                binary.append(False)

        N = len(toks)
        assert len(binary) == N, "BINARY AND TOKS DIDNT MATCH LEN"
        if direction == 'before':
            denoms = range(N, 0, -1)
        else:
            denoms = range(1, N+1, 1)
        
        weights = [(1/d if val else 0) for (val, d) in zip(binary, denoms)]
        if len(weights) == 0:
            weights = [0]
        
        return weights

    
    # Regex for extracting URLs
    url_regex = r'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*(),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+'
    
    # Find all URLs in the text
    urls = re.findall(url_regex, text)

    # Tokenize the text
    tokens = text.split()

    # Find the indices of the tokens that are URLs
    url_indices = [i for i, token in enumerate(tokens) if re.match(url_regex, token)]

    # Extract N tokens before and after each URL
    contexts = []
    for index in url_indices:
        start = max(0, index - N)  # Ensure the start is not negative
        end = min(len(tokens), index + N + 1)  # Ensure the end does not exceed the length of the tokens
        beforeTokens = tokens[start:index]
        afterTokens = tokens[index + 1:end]
        beforeWeights = getWeights(beforeTokens, 'before')
        afterWeights = getWeights(afterTokens, 'after')
        before = ' '.join(beforeTokens)
        after = ' '.join(afterTokens)
        
        contexts.append((tokens[index], before, after, max(beforeWeights), max(afterWeights)))
    
    contexts = filter(
        lambda x: max(x[3], x[4]) > 0,
        contexts
    )

    contexts = sorted(
        contexts,
        key=lambda x: x[3]+x[4],
        reverse=True
    )
    

    return contexts

In [5]:
unreadInbox = getMessages(['INBOX', 'UNREAD'])

Found 178 total unread messages in ['INBOX', 'UNREAD']
Getting message bodies...
Found 0 / 178 already existing messages in DB.


100%|██████████████████████████████████████████████████████████████████████████████████████| 178/178 [00:13<00:00, 12.77it/s]

Filtering message bodies...
FAILED TO ADD A PART: 18bf31198f61b623
[False, False, False]
FAILED TO GET BODY FOR THIS MESSAGE:
{
  "id": "18bde1f06f40db6d",
  "threadId": "18bde1f06f40db6d",
  "labelIds": [
    "CATEGORY_PROMOTIONS",
    "UNREAD",
    "INBOX"
  ],
  "snippet": "Welcome to our newslettter :)\u034f\u200c \u034f\u200c \u034f\u200c \u034f\u200c \u034f\u200c \u034f\u200c \u034f\u200c \u034f\u200c \u034f\u200c \u034f\u200c \u034f\u200c \u034f\u200c \u034f\u200c \u034f\u200c \u034f\u200c \u034f\u200c \u034f\u200c \u034f\u200c \u034f\u200c \u034f\u200c \u034f\u200c \u034f\u200c \u034f\u200c \u034f\u200c \u034f\u200c \u034f\u200c \u034f\u200c \u034f\u200c \u034f\u200c \u034f\u200c \u034f\u200c \u034f\u200c \u034f\u200c \u034f\u200c \u034f\u200c \u034f\u200c \u034f\u200c \u034f\u200c \u034f\u200c \u034f\u200c \u034f\u200c \u034f\u200c \u034f\u200c \u034f\u200c \u034f\u200c \u034f\u200c \u034f\u200c \u034f\u200c \u034f\u200c \u034f\u200c \u034f\u200c \u034f\u200c \u034f\u200c \u03

In [61]:
contexts = []
for msg in unreadInbox[:10]:
    try:
        body = msg['body']
    except KeyError:
        continue

    print(body)
    print('-' * 80)
    urlResults = extract_urls_with_context(body)
    contexts.extend(urlResults)
    print(f"Found {len(urlResults)} total urls")
    print([max(bw, aw) for _, _, _, bw, aw in urlResults])
    print()
    for item in urlResults[:3]:
        url, before, after, bweight, aweight = item
        if max(bweight, aweight) > 0:
            print(url[:80])
            print(f'\t{bweight}')
            print(f'\t{before.lower()}')
            print(f'\t{aweight}')
            print(f'\t{after.lower()}\n')
#         else:
#             m = max(bweight, aweight)
#             print(bweight, aweight, m, m>0)
    print()
    print('=' * 80)

Apple Subscription Expiring
-----------------------------------------------------------

Dear Ethan,

The following subscription will expire soon:

MuseScore Pro+ (1 year) - $99.99/year expires on January 7.

To renew or learn more, review your subscription.

Regards,
Apple

For help with subscriptions and purchases, visit Apple Support.
https://support.apple.com/billing


Apple respects your privacy.
Information regarding your personal information can be viewed at https://www.apple.com/legal/privacy/.


Terms of Sale: https://www.apple.com/legal/itunes/us/sales.html

Apple ID Summary: https://buy.itunes.apple.com/WebObjects/MZFinance.woa/wa/accountSummary?mt=8
--------------------------------------------------------------------------------
Found 2 total urls
[0.16666666666666666, 0.05]

https://support.apple.com/billing
	0.16666666666666666
	7. to renew or learn more, review your subscription. regards, apple for help with subscriptions and purchases, visit apple support.
	0
	apple res

In [9]:
print(len(contexts))
urls = []
befores = []
bWeights = []
afters = []
aWeights = []
for url, b, a, bWeight, aWeight in contexts:
    urls.append(url)
    befores.append(b if b else '<NULL>')
    afters.append(a if a else '<NULL>')
    bWeights.append(bWeight)
    aWeights.append(aWeight)

901


In [10]:
client = openai.OpenAI()

beforeResponse = client.embeddings.create(
    input=befores,
    model="text-embedding-ada-002"
)
beforeEmbeddings = [d.embedding for d in beforeResponse.data]

# afterResponse = client.embeddings.create(
#     input=afters,
#     model="text-embedding-ada-002"
# )
# afterEmbeddings = [d.embedding for d in afterResponse.data]

In [11]:
print(len(beforeEmbeddings))
print(len(beforeEmbeddings[0]))
# print(len(afterEmbeddings))
# print(len(afterEmbeddings[0]))

901
1536


In [57]:
pca = PCA(n_components=3)
# pca = PCA(n_components=2)
reduced = pca.fit_transform(beforeEmbeddings)
df = pd.DataFrame(reduced)
df['Weight'] = bWeights
df['Text'] = [val[-100:] for val in befores]
df['Contains'] = ['unsubscribe' in val.lower() for val in befores]
df['URL'] = [url[:100] for url in urls]

fig = px.scatter_3d(
# fig = px.scatter(
    df,
    x=0, y=1, z=2,
#     x=0, y=1,
    title='Befores:',
    hover_data=[
        'Weight',
        'Text',
        'URL'
    ],
    # color='Contains'
    color='Weight',
)
fig.show(renderer='browser')

In [29]:
# pca = PCA(n_components=3)
# # pca = PCA(n_components=2)
# reduced = pca.fit_transform(afterEmbeddings)
# df = pd.DataFrame(reduced)
# df['Text'] = [val[:100] for val in afters]
# df['Contains'] = ['unsubscribe' in val.lower() for val in afters]

# fig = px.scatter_3d(
# # fig = px.scatter(
#     df,
#     x=0, y=1, z=2,
# #     x=0, y=1,
#     title='Afters:',
#     hover_data=['Text'],
#     color='Contains'
# )
# fig.show(renderer='browser')